In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import csv 
import re


In [327]:
s = Service('C:/Users/Zane/Documents/Housing Webscrape/chromedriver')
driver = webdriver.Chrome(service=s)

URL = "https://www.homes.com/new-york-ny/homes-for-rent/"
driver.get(URL)

pages = int(driver.find_element(By.CLASS_NAME, "pageRange").text.split(" ")[-1])

In [30]:
rows = []
for page in range(pages):
    URL = "https://www.homes.com/new-york-ny/homes-for-rent/p" + str(page+1)
    driver.get(URL)
    houses = driver.find_elements(By.CLASS_NAME, "placard-container")
    links = []

    for house in houses:
        links.append(house.find_element(By.XPATH, './/article[@class="search-placard"]/a').get_attribute("href"))
        
    for link in links:
        driver.get(link)
        index = 0
        try:
            price = driver.find_element(By.CLASS_NAME, "property-info-price").text
        except:
            price = driver.find_elements(By.XPATH, './/span[@class="feature-rent"]/span')[0].text
            index = 2

        try:
            street = driver.find_elements(By.CLASS_NAME, "property-info-address-main")[0].text
        except:
            street = driver.find_elements(By.CLASS_NAME, "property-info-address-main--title")[0].text
            
        number_of_beds = driver.find_elements(By.XPATH, './/span[@class="feature-beds"]/span')[index].text
        number_of_bathrooms = driver.find_elements(By.XPATH, './/span[@class="feature-baths"]/span')[index].text
        house_area = driver.find_elements(By.XPATH, './/span[@class="feature-sqft"]/span')[index].text
        city_state_zipcode = driver.find_elements(By.CLASS_NAME, "property-info-address-citystatezip")[0].text
        
        data = [price, number_of_beds, number_of_bathrooms, house_area, street, city_state_zipcode]
        rows.append(data)

        driver.back()

NoSuchWindowException: Message: no such window: window was already closed
  (Session info: chrome=105.0.5195.102)
Stacktrace:
Backtrace:
	Ordinal0 [0x0028DF13+2219795]
	Ordinal0 [0x00222841+1779777]
	Ordinal0 [0x0013423D+803389]
	Ordinal0 [0x0011D646+710214]
	Ordinal0 [0x00121122+725282]
	Ordinal0 [0x00125699+743065]
	Ordinal0 [0x00135610+808464]
	Ordinal0 [0x0018E93A+1173818]
	Ordinal0 [0x0017E616+1107478]
	Ordinal0 [0x00157F89+950153]
	Ordinal0 [0x00158F56+954198]
	GetHandleVerifier [0x00582CB2+3040210]
	GetHandleVerifier [0x00572BB4+2974420]
	GetHandleVerifier [0x00326A0A+565546]
	GetHandleVerifier [0x00325680+560544]
	Ordinal0 [0x00229A5C+1808988]
	Ordinal0 [0x0022E3A8+1827752]
	Ordinal0 [0x0022E495+1827989]
	Ordinal0 [0x002380A4+1867940]
	BaseThreadInitThunk [0x75A56739+25]
	RtlGetFullPathName_UEx [0x777890AF+1215]
	RtlGetFullPathName_UEx [0x7778907D+1165]
	(No symbol) [0x00000000]


In [ ]:
fields = ["price", "number_of_beds", "number_of_bathrooms", "house_area", "street", "city_state_zipcode"] 
filename = "NY_rent_data.csv"
    
with open(filename, 'w', newline='', encoding="utf-8") as csvfile: 
    csvwriter = csv.writer(csvfile) 
    csvwriter.writerow(fields) 
    csvwriter.writerows(rows)
    
csvFile = pd.read_csv(filename)
print(csvFile)

In [115]:
title = driver.find_element(By.CLASS_NAME, "property-name")

In [116]:
title.text

'House for Rent'

In [22]:
s = Service('C:/Users/Zane/Documents/Housing Webscrape/chromedriver')
driver = webdriver.Chrome(service=s)

URL = "https://www.rentals.com/New-York/New-York-City/?page=1"
driver.get(URL)

pages = int(driver.find_elements(By.XPATH, '//div[@data-tid="pagination_page_count"]')[0].text.split(" ")[-1])

In [23]:
rows = []
for page in range(0, pages):
    URL = "https://www.rentals.com/New-York/New-York-City/?page=" + str(page+1)
    driver.get(URL)
    houses = driver.find_elements(By.XPATH, '//div[@data-tid="listing-section"]')
    links = []
    house_per_page = 31
    
    for i in range(house_per_page):
        try:
            links.append("https://www.rentals.com/New-York/New-York/" + houses[i].get_attribute("id"))
        except:
            pass

    for link in links:
        driver.get(link)
        general_info = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "._3EHgB"))).text.split("\n")

        price = general_info[0]
        
        address_headline = driver.find_element(By.XPATH, '//div[@data-tid="address_headline"]').text
        if (address_headline):
            street = address_headline
        else:
            street = driver.find_element(By.XPATH, '//div[@data-tid="property_label_headline"]').text
            
        #loop through general information to find where the city, state, and zipcode information is. Break the loop once the information is found
        for info in general_info:
            city_state_zipcode = re.findall(".*, .{2} \d{5}$",info)
            if(city_state_zipcode):
                city_state_zipcode = city_state_zipcode[0]
                break

                
        bed_bath = driver.find_element(By.XPATH, '//span[@data-tid="bed_bath_section"]').text.split(",")
        try:
            bed = bed_bath[0].strip()
        except:
            bed = "NA"
        try:
            bathroom = bed_bath[1].strip()
        except:
            bathroom = "NA"
        data = [price, bed, bathroom, street, city_state_zipcode]
        rows.append(data)
        count += 1
        
        print(data,count)
        
        price = ""
        bed = ""
        bathroom = ""
        street = ""
        city_state_zipcode = ""

341
['$2,100per month', '3 bed', '1 bath', '84 Scheerer Ave Apartments', 'Newark, NJ 07112'] 1
['$2,000per month', '3 bed', '2 bath', '465 S 15th St Apartments', 'Newark, NJ 07103'] 2
['$2,900per month', '3 bed', '2 bath', '155 65 116th Dr Apartments', 'Queens, NY 11434'] 3
['$2,250per month', '3 bed', '1 bath', '118 26 Long St Apartments', 'Queens, NY 11434'] 4
['$2,200per month', '3 bed', '1 bath', '136 Hobson St 2 Apartments', 'Newark, NJ 07112'] 5
['$2,500per month', '2 bed', '1 bath', '901 E Jersey St 321 Apartments', 'Elizabeth, NJ 07201'] 6
['$2,300per month', '2 bed', '1 bath', '257 Edison St Apartments', 'Staten Island, NY 10306'] 7
['$1,500per month', '1 bed', '1 bath', '49 Latourette Ln 1 Apartments', 'Staten Island, NY 10314'] 8
['$2,500per month', '3 bed', '1 bath', '93 32 197th St Apartments', 'Queens, NY 11423'] 9
['$1,400per month', 'Studio', '1 bath', '14 Dartmouth Loop BSMNT Apartments', 'Staten Island, NY 10306'] 10
['$2,600per month', '2 bed', '1 bath', '39 14 214th

['$1,725per month', '1 bed', '1 bath', '685 Prospect Ave 4 A Apartments', 'Fairview, NJ 07022'] 28
['$2,000per month', '2 bed', '1 bath', '209 Montauk Ave 2 B Apartments', 'Brooklyn, NY 11208'] 29
['$2,600per month', '3 bed', '1 bath', '455 Milford St Apartments', 'Brooklyn, NY 11208'] 30
344
['$2,800per month', 'Studio', '0.5 bath', '51 20 103rd St Apartments', 'Queens, NY 11368'] 1
['$2,500per month', '3 bed', '1 bath', '495 Lincoln Ave 3 Apartments', 'Brooklyn, NY 11208'] 2
['$4,800per month', 'Studio', '1 bath', '106 16 Corona Ave Apartments', 'Queens, NY 11368'] 3
['$1,700per month', '1 bed', '1 bath', '108 15 37th Dr 1 D Apartments', 'Queens, NY 11368'] 4
['$2,100per month', '3 bed', '2 bath', '92 Oak St 2 Apartments', 'Bayonne, NJ 07002'] 5
['$2,650per month', '2 bed', '1 bath', '5314 Avenue O 3 Apartments', 'Brooklyn, NY 11234'] 6
['$2,200per month', '2 bed', '1 bath', '86 22 111 Th St 2 Apartments', 'Queens, NY 11418'] 7
['$2,350per month', '2 bed', '1 bath', '89 17 Desarc Rd 

['$3,200per month', '1 bed', '1 bath', '151 Manhattan Ave 3 A Apartments', 'Brooklyn, NY 11206'] 24
['$2,800per month', '2 bed', '1 bath', '1036 Willow Ave 5 Apartments', 'Hoboken, NJ 07030'] 25
['$3,750per month', '2 bed', '1 bath', '1017 Jefferson St 314 Apartments', 'Hoboken, NJ 07030'] 26
['$1,550per month', '1 bed', '1 bath', '65 Summit Ave 401 Apartments', 'Jersey City, NJ 07304'] 27
['$1,375per month', '1 bed', '1 bath', '526 29th St 3 Apartments', 'Union City, NJ 07087'] 28
['$1,700per month', '1 bed', '1 bath', '1901 John F Kennedy Blvd 112 Apartments', 'Union City, NJ 07087'] 29
['$4,250per month', '3 bed', '1.5 bath', '35 Kane Pl 1 Apartments', 'Brooklyn, NY 11233'] 30
347
['$2,300per month', '2 bed', '2 bath', '144 Bergen Ave 2 A Apartments', 'Jersey City, NJ 07305'] 1
['$4,000per month', '3 bed', '2.5 bath', '7004 Boulevard E 39 D Apartments', 'Guttenberg, NJ 07093'] 2
['$2,600per month', '2 bed', '1 bath', '62 86 60th Dr Apartments', 'Queens, NY 11378'] 3
['$2,450per mont

['$1,800per month', '1 bed', '1 bath', '50 N 10th Ave Apartments', 'Mount Vernon, NY 10550'] 20
['$2,000per month', '1 bed', '1 bath', '1538 Lydia Ave Apartments', 'Elmont, NY 11003'] 21
['$2,300per month', '2 bed', '1 bath', '510 Midland Ave 3 H Apartments', 'Yonkers, NY 10704'] 22
['$2,400per month', '2 bed', '1 bath', '58 Halley St Apartments', 'Yonkers, NY 10704'] 23
['$2,295per month', '1 bed', '1.5 bath', '703 Pelham Rd 106 Apartments', 'New Rochelle, NY 10805'] 24
['$2,400per month', '2 bed', '1 bath', '5 7th St Apartments', 'Valley Stream, NY 11581'] 25
['$1,750per month', '1 bed', '1 bath', '1300 Fulton St 5 Apartments', 'Rahway, NJ 07065'] 26
['$1,850per month', '1 bed', '1 bath', '1300 Fulton St 3 Apartments', 'Rahway, NJ 07065'] 27
['$2,200per month', '3 bed', '1.5 bath', '85 Jefferson St Apartments', 'Paterson, NJ 07522'] 28
['$3,975per month', '4 bed', '2.5 bath', '3 Tanglewood Dr Apartments', 'Livingston, NJ 07039'] 29
['$1,800per month', '3 bed', '1 bath', '297 W Broadw

['$2,000per month', '1 bed', '1.5 bath', '25 Mohawk Ave Apartments', 'East Atlantic Beach, NY 11561'] 16
['$4,000per month', '4 bed', '2 bath', '30 I U Willets Rd Apartments', 'Albertson, NY 11507'] 17
['$4,650per month', '3 bed', '2 bath', '568 Laurelton Blvd Apartments', 'Long Beach, NY 11561'] 18
['$3,500per month', '2 bed', '1.5 bath', '259 Collignon Way 2 B Apartments', 'River Vale, NJ 07675'] 19
['$3,500per month', '3 bed', '2.5 bath', '80 Tyrrell St 1 Apartments', 'Staten Island, NY 10307'] 20
['$2,150per month', '1 bed', '1 bath', '513 Willow Ave 1 Apartments', 'Hoboken, NJ 07030'] 21
['$2,200per month', '1 bed', '1 bath', '939 Park Ave Apartments', 'Hoboken, NJ 07030'] 22
['$1,500per month', '1 bed', '1 bath', '3202 John F Kennedy Blvd B Apartments', 'Jersey City, NJ 07306'] 23
['$1,800per month', '1 bed', '1 bath', '22 15 Steinway St 4 Apartments', 'Queens, NY 11105'] 24
['$1,550per month', 'Studio', '1 bath', '84 11 Elmhurst Ave 2 Apartments', 'Queens, NY 11373'] 25
['$1,500

['$1,950per month', '1 bed', '1 bath', '112 Mc Whorter St 1 Apartments', 'Newark, NJ 07105'] 12
['$1,950per month', '1 bed', '1 bath', '112 Mc Whorter St 2 Apartments', 'Newark, NJ 07105'] 13
['$2,200per month', '2 bed', '1 bath', '68 Christie St Apartments', 'Ridgefield Park, NJ 07660'] 14
['$2,200per month', '3 bed', '1 bath', '140 84 34th Ave 3 FL Apartments', 'Queens, NY 11354'] 15
['$1,700per month', '1 bed', '1 bath', '2337 Hudson Terrace E 19 Apartments', 'Fort Lee, NJ 07024'] 16
['$2,700per month', '3 bed', '1 bath', '32 Lester Ct 1 Apartments', 'Brooklyn, NY 11229'] 17
['$3,000per month', '3 bed', '2.5 bath', '191 Harrison St 1 Apartments', 'Bloomfield, NJ 07003'] 18
['$1,400per month', '1 bed', '1 bath', '869 S 14th St Apartments', 'Newark, NJ 07108'] 19
['$1,850per month', '2 bed', '1 bath', '235 Monroe St 2 A Apartments', 'Passaic, NJ 07055'] 20
['$1,700per month', '2 bed', '1 bath', '253 Palisade Ave 5 Apartments', 'Garfield, NJ 07026'] 21
['$3,850per month', '3 bed', '2 b

['$1,950per month', '2 bed', '1 bath', '61 35 218th St 3RD Apartments', 'Queens, NY 11364'] 8
['$1,950per month', '2 bed', '1 bath', '100 42 Francis Lewis Blvd 1ST Apartments', 'Queens, NY 11429'] 9
['$2,200per month', '2 bed', '1 bath', '705 Midland Ave 2 Apartments', 'Garfield, NJ 07026'] 10
['$3,000per month', '3 bed', '1 bath', '20 Evelyn Ct Apartments', 'Maplewood, NJ 07040'] 11
['$1,750per month', '1 bed', '1 bath', '89 11 Pontiac St 1 Apartments', 'Queens, NY 11427'] 12
['$6,000per month', '4 bed', '4.5 bath', '92 Franklin St Apartments', 'Tenafly, NJ 07670'] 13
['$2,400per month', '3 bed', '1 bath', '89 51 241st St 2ND Apartments', 'Queens, NY 11426'] 14
['$1,200per month', 'Studio', '1 bath', '140 12 243rd St 2ND Apartments', 'Queens, NY 11422'] 15
['$3,200per month', '4 bed', '1.5 bath', '93 14 242nd St Apartments', 'Queens, NY 11426'] 16
['$3,000per month', '4 bed', '2 bath', '251 43 57th Ave Apartments', 'Queens, NY 11362'] 17
['$2,100per month', '3 bed', '1 bath', '247 29 

In [25]:
fields = ["price", "number_of_beds", "number_of_bathrooms", "street", "city_state_zipcode"] 
filename = "9-11-2022 NY Rental Data.csv"
    
with open(filename, 'a', newline='', encoding="utf-8") as csvfile: 
    csvwriter = csv.writer(csvfile) 
    #csvwriter.writerow(fields) 
    csvwriter.writerows(rows)
    
csvFile = pd.read_csv(filename)
print(csvFile)


                  price number_of_beds number_of_bathrooms  \
0       $3,600per month         Studio              1 bath   
1      $15,500per month          2 bed              2 bath   
2       $8,571per month          3 bed              2 bath   
3       $8,249per month          3 bed              2 bath   
4       $8,624per month          3 bed              2 bath   
...                 ...            ...                 ...   
13701   $4,950per month          2 bed              2 bath   
13702   $6,240per month          3 bed              2 bath   
13703   $5,500per month          2 bed              2 bath   
13704   $4,100per month          1 bed              1 bath   
13705   $4,500per month          1 bed              1 bath   

                          street  city_state_zipcode  
0      62 Madison Ave Apartments  New York, NY 10016  
1               220 Front Street  New York, NY 10038  
2               70 W 45th St 43B  New York, NY 10036  
3                15 W 55th St 7A  N

In [8]:
driver.get("https://www.rentals.com/New-York/New-York/lv208420077/")

In [5]:
last_updated = driver.find_element(By.XPATH, '//div[@data-tid="last_updated_time"]').text

In [7]:
if(last_updated):
    print(1)
else:
    print(2)

2


In [8]:
general_info = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "._3EHgB"))).text.split("\n")

In [29]:
city_state_zipcode = general_info[-3]

In [54]:
for info in general_info:
    if info[:9] == "New York,":
        city_state_zipcode = info

$5,500per
450 W 42n
New York,
2 bed, 1 
Updated 1


In [55]:
city_state_zipcode

'New York, NY 10036'

In [12]:
bed_bath

['Studio']

In [15]:
try:
    bed = bed_bath[0].strip()
except:
    bed = "NA"
try:
    bathroom = bed_bath[1].strip()
except:
    bathroom = "NA"

In [42]:
general_info = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "._3EHgB"))).text.split("\n")

In [58]:
#loop through general information to find where the city, state, and zipcode information is. Break the loop once the information is found
for info in general_info:
    city_state_zipcode = re.findall(".*, .{2} \d{5}$",info)
    if(city_state_zipcode):
        break


In [51]:
x = re.findall(".*, .{2} \d{5}$",general_info[2])
print(x)

['New York, NY 10038']


In [69]:
        price = ""
        bed = ""
        bathroom = ""
        street = ""
        city_state_zipcode = ""